In [107]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from splinter import Browser
import csv
import warnings
warnings.filterwarnings('ignore')

In [108]:
#url = 'https://www.remax.ca/ab/calgary-real-estate'
province_abb = input("Please enter the abbreviation of your province: ")
province_name = province_abb.lower()
city_name_input = input("Please enter city: ")
city_name = city_name_input.lower()
base_url = f'https://www.remax.ca/{province_name}/{city_name}-real-estate'
print(base_url)

Please enter the abbreviation of your province: ab
Please enter city: Calgary
https://www.remax.ca/ab/calgary-real-estate


In [109]:
house_address = []
house_details = []


base_url = f'https://www.remax.ca/{province_name}/{city_name}-real-estate?page='
urls = [base_url + str(x) for x in range(1,2)]


for url in urls:
    
    #browser.visit(url)
#     time.sleep(5)

    # Parse HTML with Beautiful Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    time.sleep(5)

    addresses = soup.find_all('div', class_='left-content flex-one')
    for address in addresses:
        house_address.append(address.text)
        
    details = soup.find_all('div', class_='property-details')
    for detail in details:
        house_details.append(detail.text)
  

In [117]:
address_df = pd.DataFrame(house_address)

new_df = address_df[0].str.split(' ', 2, expand=True)
new_df["price"] = new_df[1].str.replace("$", "")

del new_df[0]
del new_df[1]
new_df.head()

,2,price
0,"140 CITADEL CREST CIR NW, Calgary, AB, T3G 4G3","489,000"
1,"79 GALBRAITH DR SW, Calgary, AB, T3E 4Z5","351,999"
2,"275 TARAVISTA ST NE, Calgary, AB, T3J 4S5","449,999"
3,"25 DOUGLASBANK RISE SE, Calgary, AB, T2Z 2C5","469,900"
4,"111 PANORAMA HILLS PL NW, Calgary, AB, T3K 4R9","419,900"


In [160]:
final_df = new_df[2].str.split(', Calgary, AB, ', expand=True)
final_df.head()

,0,1
0,140 CITADEL CREST CIR NW,T3G 4G3
1,79 GALBRAITH DR SW,T3E 4Z5
2,275 TARAVISTA ST NE,T3J 4S5
3,25 DOUGLASBANK RISE SE,T2Z 2C5
4,111 PANORAMA HILLS PL NW,T3K 4R9


In [124]:
df_add = pd.concat([new_df, final_df], axis=1)
del df_add[2]
df_add.columns = ["price", "address", "postal_code"]
df_add.head()

,price,address,postal_code
0,"489,000",140 CITADEL CREST CIR NW,T3G 4G3
1,"351,999",79 GALBRAITH DR SW,T3E 4Z5
2,"449,999",275 TARAVISTA ST NE,T3J 4S5
3,"469,900",25 DOUGLASBANK RISE SE,T2Z 2C5
4,"419,900",111 PANORAMA HILLS PL NW,T3K 4R9


In [125]:
details = pd.DataFrame(house_details)

details_df = details[0].str.split('|', expand=True)
details_df

del details_df[2]

details_df.columns = ["bedrooms", "bath", "property_type"]
details_df.head()

,bedrooms,bath,property_type
0,3 bed,2 + 1 bath,house
1,N/A bed,N/A bath,house
2,5 bed,2 + 1 bath,house
3,4 bed,2 + 1 bath,house
4,3 bed,2 bath,house


In [126]:

calgary_df = pd.concat([df_add, details_df], axis=1)
calgary_df.head()

,price,address,postal_code,bedrooms,bath,property_type
0,"489,000",140 CITADEL CREST CIR NW,T3G 4G3,3 bed,2 + 1 bath,house
1,"351,999",79 GALBRAITH DR SW,T3E 4Z5,N/A bed,N/A bath,house
2,"449,999",275 TARAVISTA ST NE,T3J 4S5,5 bed,2 + 1 bath,house
3,"469,900",25 DOUGLASBANK RISE SE,T2Z 2C5,4 bed,2 + 1 bath,house
4,"419,900",111 PANORAMA HILLS PL NW,T3K 4R9,3 bed,2 bath,house


In [141]:
calgary_df.to_csv('calgary_df.csv', encoding="utf-8")

----

# Retrieving Walkscore for the postal codes in Calgary DataFrame

post_code_list = []

for i in calgary_df["postal_code"]:
    post_code_list.append(i)
    
post_code_list_3 = post_code_list[0:2]
post_code_list_3

In [142]:
score_list = []

for i in calgary_df["postal_code"]:
    try:
        postal_code = i.replace(" ", "%20")
        url = "https://www.walkscore.com/score/" + str(postal_code)

        executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
        browser = Browser("chrome", **executable_path, headless = True)

        browser.visit(url)
    #     time.sleep(2)
    

        # HTML Object
        code_html = browser.html

        # Parse HTML with Beautiful Soup
        code_soup = BeautifulSoup(code_html, 'html.parser')
        time.sleep(4)

        walk_score1 = code_soup.find_all('div', class_='clearfix score-div')
        time.sleep(1)

        walk_score_list = []

        for j in walk_score1:
            walk_score_list.append(j.img['alt'])

            scores = []

            for k in walk_score_list:
                number = k.split(" ")[0]
                scores.append(number)


        score_list.append(scores)
        
    except AttributeError:
        empty = []
        score_list.append(empty)
  

In [161]:
# print(score_list)

In [144]:
len(score_list)

40

In [145]:
score_df = pd.DataFrame(score_list, columns=['walk_score', 
                                             'transit_score', 
                                             'bike_score'])
score_df.head()

,walk_score,transit_score,bike_score
0,23,38,65
1,43,47,67
2,10,49,62
3,22,41,65
4,37,48,71


In [146]:
postal_code_df = pd.DataFrame(post_code_list, columns=['postal_code'])
merged_df = pd.concat([postal_code_df, score_df], axis=1)
merged_df.head()

,postal_code,walk_score,transit_score,bike_score
0,T3G 4G3,23,38,65
1,T3E 4Z5,43,47,67
2,T3J 4S5,10,49,62
3,T2Z 2C5,22,41,65
4,T3K 4R9,37,48,71


In [159]:
calgary_df.to_csv('score_df.csv', encoding="utf-8")

In [147]:
calgary_df.head()

,price,address,postal_code,bedrooms,bath,property_type
0,"489,000",140 CITADEL CREST CIR NW,T3G 4G3,3 bed,2 + 1 bath,house
1,"351,999",79 GALBRAITH DR SW,T3E 4Z5,N/A bed,N/A bath,house
2,"449,999",275 TARAVISTA ST NE,T3J 4S5,5 bed,2 + 1 bath,house
3,"469,900",25 DOUGLASBANK RISE SE,T2Z 2C5,4 bed,2 + 1 bath,house
4,"419,900",111 PANORAMA HILLS PL NW,T3K 4R9,3 bed,2 bath,house


In [154]:
from sqlalchemy import create_engine

In [157]:
rds_connection_string = "postgres:123@localhost:5432/realestate_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [158]:
engine.table_names()

[]

In [29]:
calgary_df.to_sql(name= "calgary_df", 
                      con=engine, 
                      if_exists="append", 
                      index=False)